In [ ]:
%matplotlib inline
import atomap.api as am
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
import hyperspy.api as hs
filename='HRSTEM_PbTe_processed.dm4'
image = hs.load(filename) 

In [ ]:
# normalize the data
norm = np.linalg.norm(image.data)       # To find the norm of the array
                                        # Printing the value of the norm
normalized_array = image.data/norm      
normalized_array = normalized_array-np.min(normalized_array)

In [ ]:
# assign the normalized array
image.data = normalized_array

In [ ]:
image.plot(colorbar=False,axes_off=True,title='')
#plt.savefig('PbTe_raw.tif',dpi=300,bbox_inches='tight', pad_inches=0)

In [ ]:
# test the parameter for separating columns
s_peaks = am.get_feature_separation(image, separation_range=(13, 20), show_progressbar=True, pca=True,threshold_rel=0.05)
s_peaks.plot()

In [ ]:
A_positions = am.get_atom_positions(image, separation=13, pca=True, threshold_rel = 0.05)
#number of detected atoms (atom columns)
print(len(A_positions))
print(A_positions.shape)

In [ ]:
sublattice = am.Sublattice(A_positions, image=image.data, color = 'r')
sublattice

In [ ]:
sublattice.find_nearest_neighbors()
sublattice.refine_atom_positions_using_center_of_mass()

In [ ]:
sublattice.refine_atom_positions_using_2d_gaussian()

In [ ]:
sublattice.get_position_history().plot()

In [ ]:
# check the sublattice after refinement
sublattice.plot()

In [ ]:
sublattice.construct_zone_axes(atom_plane_tolerance=0.5)
sublattice

In [ ]:
sublattice.plot_planes()

In [ ]:
from atomap.tools import remove_integrated_edge_cells
i_points_A, i_record_A, p_record_A = sublattice.integrate_column_intensity()
i_points_A, i_record_A, p_record_A = remove_integrated_edge_cells(i_points_A, i_record_A, p_record_A, edge_pixels=10)
i_record_A.plot(cmap='viridis')

In [ ]:
#Limit the radius of Voronoid cells
#sub_A = ? elements?
x_A, y_A = sublattice.x_position, sublattice.y_position
i_points_A, i_record_A, p_record_A = am.integrate(sublattice.image, x_A, y_A, max_radius= 23) 


In [ ]:
i_record_A.plot(cmap = 'viridis')

In [ ]:
# renormalize intensity
i_points_A_norm = (i_points_A-np.min(i_points_A))/(np.max(i_points_A)-np.min(i_points_A))*100

In [ ]:
# create customized diverging color map

my_gradient = LinearSegmentedColormap.from_list('my_gradient', (
    # Edit this gradient at https://eltos.github.io/gradient/#0:2D0076-35:7B55B9-50:FFFFFF-65:FFD78E-100:FFE65F
    (0.000, (0.176, 0.000, 0.463)),
    (0.350, (0.482, 0.333, 0.725)),
    (0.500, (1.000, 1.000, 1.000)),
    (0.650, (1.000, 0.843, 0.557)),
    (1.000, (1.000, 0.902, 0.373))))

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

sc = ax.scatter(x_A, y_A, s = 150, c=i_points_A_norm, cmap=my_gradient,vmin=0,vmax=100,alpha=0.7)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.3)
cbar=plt.colorbar(sc, cax=cax)
cbar.set_ticks([0,25,50,75,100])
cbar.ax.tick_params(labelsize=20,colors='w')
for l in cbar.ax.yaxis.get_ticklabels():
    l.set_weight('bold')
    l.set_fontsize(25)
ax.imshow(image,cmap='gray')
ax.axis('off')
plt.show()